<a href="https://colab.research.google.com/github/manavmittal05/Large-Language-Models/blob/main/A1/Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install sentence-transformers==2.2.2
!pip install bitsandbytes
!pip install datasets
!pip install -qU langchain-huggingface
!pip install -qU "langchain-chroma>=0.1.2"
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 636.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-fm2bhw80
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-b

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import os
import random
import string
import warnings
from datasets import load_dataset
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

warnings.filterwarnings('ignore')

In [6]:
#embedding model
embedding_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
    # model_kwargs={'device':'cuda'},
    # encode_kwargs={'device': 'cuda', 'batch_size': 16}
)

In [8]:
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embedding_model,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)

In [9]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="Sir Keir Starmer became Prime Minister on 5 July 2024. The Prime Minister is the leader of His Majesty’s Government and is ultimately responsible for the policy and decisions of the government.",
    metadata={"source": "website"},
    id=1,
)

document_2 = Document(
    page_content="Question: One side of the bottom layer of a triangular pyramid has 11 balls. How many are there in the whole pyramid? Note that the pyramid is equilateral and solid.\nAnswer: As there are 11 balls along one side, it means that there are 11 layers of balls. The top most layer has 1 ball. The second layer has 3 (1+2) balls. The third layer has 6 (1+2+3) balls. The fourth layer has 10 (1+2+3+4) balls. The fifth layer has 15 (1+2+3+4+5) balls. Similarly, there are 21, 28, 36, 45, 55 and 66 balls in the remaining layers. Hence, the total number of balls is = 1 + 3 + 6 + 10 + 15 + 21 + 28 + 36 + 45 + 55 + 66 = 286 balls",
    metadata={"source": "website"},
    id=2,
)

document_3 = Document(
    page_content="Question: When a number is divided by 5, the remainder is 3. What is the remainder when twice the number is divided by 5? Answer: 1",
    metadata={"source": "website"},
    id=3,
)

fact_check1 = Document(
    page_content="Muhammad Yunus, 2006 Nobel Peace Prize winner, has been appointed chief adviser to Bangladesh’s interim government following Prime Minister Sheikh Hasina’s resignation amid protests. Yunus, born in 1940, is an economist known for pioneering microcredit through Grameen Bank, founded in 1983. The bank has loaned over $34 billion to nearly 10 million people, with a 97% repayment rate, inspiring global microfinance initiatives. Yunus’s appointment aims to address unrest over government job quotas.",
    metadata={"source": "news"},
    id=4,
)

fact_check2 = Document(
    page_content="Indian wrestler Vinesh Phogat, 29, announced her retirement after being disqualified from the Olympics for being 100 grams overweight before her gold medal bout in the 50 kg category. She appealed to the Court of Arbitration for Sports (CAS) to be awarded a joint silver medal, citing severe dehydration from extreme weight-cutting measures. Phogat, a three-time Olympian in different weight classes, became the first international wrestler to defeat reigning champion Yui Susaki and the first Indian female wrestler to reach an Olympic final in 2024. She is a multiple Commonwealth Games gold medalist and has won two World Wrestling Championship bronze medals.",
    metadata={"source": "news"},
    id=5,
)

fact_check3 = Document(
    page_content="Archaeologists have discovered a terracotta pipeline during the 10th phase of excavation at Keeladi, a Sangam Era site in Tamil Nadu. This find highlights advanced water management practices from over 2,600 years ago. Keeladi, located near Madurai, has revealed over 20,000 artifacts, suggesting the Sangam Era began around 600 BCE, earlier than previously thought. Additional discoveries include rice husks from 3,200 years ago and evidence of early iron technology.",
    metadata={"source": "news"},
    id=6,
)

documents = [
    document_1,
    document_2,
    document_3,
    fact_check1,
    fact_check2,
    fact_check3
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['2c2c383a-de3a-420e-ac45-1c252bfafa1f',
 'e9bba9df-c446-45a2-bc1b-94f7841f4b5d',
 '1b31589b-0ef1-41b9-a2ef-5c998f829e5d',
 'b50a24ea-4075-4c49-8c8d-e53e134bda6c',
 '8e3230fe-60a7-4129-aea7-422ee72284bb',
 '04769d31-4584-4301-b4cf-604f402426e1',
 '3bbe8c6b-3ed2-4c4a-a7fc-f49aca8b42ea',
 '9edbb3bf-46d6-4ef5-a56a-daa275c42655',
 '3fdc3b46-afe4-42a8-86ae-8c79bb01f143',
 '30457669-6c08-45ea-ba5f-ad0639f8e6fa']

In [11]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.916557] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [12]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


### Llama3.1 Prompting and Setup

In [ ]:
from huggingface_hub import login
from google.colab import userdata

# Login with your API token
login(userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.bfloat16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Fact checking using llama3.1

In [ ]:
messages = [
    {"role": "user", "content": "Select two random numbers between 1381 and 1453 and multiply them together, reporting the result. "},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


I've randomly selected two numbers between 1381 and 1453:

Number 1: 1412
Number 2: 1419

Multiplying them together gives:

1412 × 1419 = 20,049,288


In [ ]:
messages = [
    {"role": "user", "content": "what the current price of gold in INR"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The current price of gold in Indian Rupees (INR) can vary depending on the market conditions, global demand, and other factors. As of [current date], the price of gold in INR is:

* 10 grams of 22-karat gold: ₹45,500 - ₹46,500
* 10 grams of 24-karat gold: ₹48,500 - ₹49,500
* 1 gram of 24-karat gold: ₹4,850 - ₹4,950

Please note that these prices are approximate and can fluctuate depending on the market conditions. It's always a good idea to check with a reliable gold dealer or a financial website for the latest prices.

Some popular gold prices in INR can be found on websites such as:

* Gold Price India (goldpriceindia.com)
* A24M (a24m.com)
* Gold Rate Today (goldratetoday.com)
* Moneycontrol (moneycontrol.com)

Please keep in mind that these prices are subject to change and may not reflect the actual price at the time of your inquiry.


In [ ]:
messages = [
    {"role": "user", "content": "which is bigger, 9.11 and 9.9"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


A simple question!

9.11 is bigger than 9.9.


### OpenHathi Prompting and Setup

In [ ]:
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

In [ ]:
# loading the model from huggingface
model = AutoModelForCausalLM.from_pretrained(
    'sarvamai/OpenHathi-7B-Hi-v0.1-Base',
    load_in_4bit=True,                                                            # loading the model with 4 bit precision, so as to show the example on Colab GPU
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base')
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.8,
    repetition_penalty = 1.5,
    max_new_tokens = 1
)

In [ ]:
inputs = tokenizer("I'll bet the video game is alot more fun than the film.\Sentiment:", return_tensors="pt")
print(inputs)
input_ids = inputs["input_ids"].to('cuda')

{'input_ids': tensor([[    1,   306, 29915,   645,  1010,   278,  4863,  3748,   338,   394,
           327,   901,  2090,  1135,   278,  2706,  7790, 29903,   296,  2073,
         29901]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config)

In [ ]:
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

I'll bet the video game is alot more fun than the film.\Sentiment: negative


In [ ]:
import pandas as pd

splits = {'train': 'pair-class/train-00000-of-00001.parquet', 'dev': 'pair-class/dev-00000-of-00001.parquet', 'test': 'pair-class/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/sentence-transformers/all-nli/" + splits["train"])

df.head()

,premise,hypothesis,label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,1
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",2
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",0
3,Children smiling and waving at camera,They are smiling at their parents,1
4,Children smiling and waving at camera,There are children present,0
